In [3]:
import torch
import random

In [9]:
vocab = [char for char in 'ABCDEFGHIJKLMNOPQESTUVWXYZ-']
vocab[0]

'A'

In [14]:
def encryptstr(originalword):
    encryptedstr = ''
    encryptedlist = [ vocab[ (vocab.index(c) + 13) % 27 ] for c in originalword ]
    encryptedstr = ''.join(encryptedlist)
    return encryptedstr 

In [16]:
ans = encryptstr(['A','B','C'])
ans

'NOP'

In [41]:
# Preparing data set 
max_len_seq = 4
number_sequences = 129

# pairs of originalword and encrypted word

dataset = [] ;

for i in range(number_sequences):
    orgwordidx = []
    orgwordidx = [random.randint(0,26) for loop in range(0,max_len_seq)]
    encwordidx = [ ( (index + 13) % 27 ) for index in orgwordidx]
    dataset.append([torch.tensor(orgwordidx),torch.tensor(encwordidx)])

In [42]:
dataset

[[tensor([23,  1, 22, 22]), tensor([ 9, 14,  8,  8])],
 [tensor([13, 12, 22,  1]), tensor([26, 25,  8, 14])],
 [tensor([ 6, 26, 17,  2]), tensor([19, 12,  3, 15])],
 [tensor([ 0, 13, 14, 25]), tensor([13, 26,  0, 11])],
 [tensor([ 7, 13,  7, 26]), tensor([20, 26, 20, 12])],
 [tensor([ 4, 16,  4, 15]), tensor([17,  2, 17,  1])],
 [tensor([ 8, 25, 10, 24]), tensor([21, 11, 23, 10])],
 [tensor([17,  0,  4, 25]), tensor([ 3, 13, 17, 11])],
 [tensor([21, 21, 12,  8]), tensor([ 7,  7, 25, 21])],
 [tensor([18, 16,  4,  9]), tensor([ 4,  2, 17, 22])],
 [tensor([18, 24, 18, 16]), tensor([ 4, 10,  4,  2])],
 [tensor([ 5, 24, 21,  7]), tensor([18, 10,  7, 20])],
 [tensor([12,  5, 11,  9]), tensor([25, 18, 24, 22])],
 [tensor([18, 13, 23, 23]), tensor([ 4, 26,  9,  9])],
 [tensor([ 2, 11,  4, 12]), tensor([15, 24, 17, 25])],
 [tensor([18,  8, 24, 16]), tensor([ 4, 21, 10,  2])],
 [tensor([15, 10,  0, 16]), tensor([ 1, 23, 13,  2])],
 [tensor([25, 24,  0, 10]), tensor([11, 10, 13, 23])],
 [tensor([

In [59]:
org,encr = dataset[0]

In [60]:
org

tensor([23,  1, 22, 22])

In [45]:
#build the network
embedding_dim = 5
hidden_dim = 10
vocab_size = len(vocab)

embed = torch.nn.Embedding(vocab_size, embedding_dim)
lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
linear = torch.nn.Linear(hidden_dim, vocab_size)
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(embed.parameters()) +
                             list(lstm.parameters()) +
                             list(linear.parameters()), lr=0.001)

In [46]:
encryptest = torch.tensor([4,5,0,25,4,0])

print(embed(encryptest))

In [65]:
# run the epochs
for epoch in range(15):
    total_loss = 0
    total_correct = 0
    for original,encrypted in dataset:
        #lstm
        #input of shape (seq_len, batch, input_size)
        #h_0 of shape (num_layers * num_directions, batch, hidden_size)
        #output of shape (seq_len, batch, num_directions * hidden_size):
        #h_n of shape (num_layers * num_directions, batch, hidden_size)
        lstm_in = embed(original)
        #adding batch dimension
        lstm_in = lstm_in.unsqueeze(1)
        lstm_out,lstm_hidden  = lstm(lstm_in,(torch.zeros(1,1,10) , torch.zeros(1,1,10) ) )
        #lstm_out.shape is 4*1*10
        scores = linear(lstm_out)
        print(original)
        scores = scores.transpose(1, 2)
        print(scores.argmax(dim=1))
        print(scores.argmax(dim=1).shape)
        encrypted = encrypted.unsqueeze(1)
        print(encrypted)
        loss = loss_fn(scores, encrypted)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print("epoch:",epoch,"loss:" , total_loss)

tensor([23,  1, 22, 22])
tensor([[ 9],
        [14],
        [ 8],
        [ 8]])
torch.Size([4, 1])
tensor([[ 9],
        [14],
        [ 8],
        [ 8]])
tensor([13, 12, 22,  1])
tensor([[26],
        [25],
        [ 8],
        [14]])
torch.Size([4, 1])
tensor([[26],
        [25],
        [ 8],
        [14]])
tensor([ 6, 26, 17,  2])
tensor([[19],
        [12],
        [ 3],
        [15]])
torch.Size([4, 1])
tensor([[19],
        [12],
        [ 3],
        [15]])
tensor([ 0, 13, 14, 25])
tensor([[13],
        [26],
        [ 0],
        [11]])
torch.Size([4, 1])
tensor([[13],
        [26],
        [ 0],
        [11]])
tensor([ 7, 13,  7, 26])
tensor([[20],
        [26],
        [20],
        [12]])
torch.Size([4, 1])
tensor([[20],
        [26],
        [20],
        [12]])
tensor([ 4, 16,  4, 15])
tensor([[17],
        [ 2],
        [17],
        [ 1]])
torch.Size([4, 1])
tensor([[17],
        [ 2],
        [17],
        [ 1]])
tensor([ 8, 25, 10, 24])
tensor([[21],
        [11],

tensor([ 2, 26,  7, 18])
tensor([[15],
        [12],
        [20],
        [ 4]])
torch.Size([4, 1])
tensor([[15],
        [12],
        [20],
        [ 4]])
tensor([10, 13,  3, 26])
tensor([[23],
        [26],
        [16],
        [12]])
torch.Size([4, 1])
tensor([[23],
        [26],
        [16],
        [12]])
tensor([26, 24, 24,  4])
tensor([[12],
        [10],
        [10],
        [17]])
torch.Size([4, 1])
tensor([[12],
        [10],
        [10],
        [17]])
tensor([18, 25,  7,  5])
tensor([[ 4],
        [11],
        [20],
        [18]])
torch.Size([4, 1])
tensor([[ 4],
        [11],
        [20],
        [18]])
tensor([14, 17,  0, 14])
tensor([[ 0],
        [ 3],
        [13],
        [ 0]])
torch.Size([4, 1])
tensor([[ 0],
        [ 3],
        [13],
        [ 0]])
tensor([14, 26, 15, 26])
tensor([[ 0],
        [12],
        [ 1],
        [12]])
torch.Size([4, 1])
tensor([[ 0],
        [12],
        [ 1],
        [12]])
tensor([ 1, 20,  8,  2])
tensor([[14],
        [ 6],

tensor([[18],
        [17],
        [ 2],
        [12]])
torch.Size([4, 1])
tensor([[18],
        [17],
        [ 2],
        [12]])
tensor([13,  8, 11, 23])
tensor([[26],
        [21],
        [24],
        [ 9]])
torch.Size([4, 1])
tensor([[26],
        [21],
        [24],
        [ 9]])
tensor([ 9, 23, 21,  8])
tensor([[22],
        [ 9],
        [ 7],
        [21]])
torch.Size([4, 1])
tensor([[22],
        [ 9],
        [ 7],
        [21]])
tensor([ 3, 12,  8,  1])
tensor([[16],
        [25],
        [21],
        [14]])
torch.Size([4, 1])
tensor([[16],
        [25],
        [21],
        [14]])
tensor([1, 3, 4, 4])
tensor([[14],
        [16],
        [17],
        [17]])
torch.Size([4, 1])
tensor([[14],
        [16],
        [17],
        [17]])
tensor([12, 21,  5,  8])
tensor([[25],
        [ 7],
        [18],
        [21]])
torch.Size([4, 1])
tensor([[25],
        [ 7],
        [18],
        [21]])
tensor([10, 23, 17,  2])
tensor([[23],
        [ 9],
        [ 3],
        [15]])

tensor([13, 13, 17, 11])
tensor([[26],
        [26],
        [ 3],
        [24]])
torch.Size([4, 1])
tensor([[26],
        [26],
        [ 3],
        [24]])
tensor([ 7, 23,  6,  3])
tensor([[20],
        [ 9],
        [19],
        [16]])
torch.Size([4, 1])
tensor([[20],
        [ 9],
        [19],
        [16]])
tensor([24, 18, 10, 14])
tensor([[10],
        [ 4],
        [23],
        [ 0]])
torch.Size([4, 1])
tensor([[10],
        [ 4],
        [23],
        [ 0]])
tensor([16, 25, 22,  7])
tensor([[ 2],
        [11],
        [ 8],
        [20]])
torch.Size([4, 1])
tensor([[ 2],
        [11],
        [ 8],
        [20]])
tensor([15,  9, 25, 18])
tensor([[ 1],
        [22],
        [11],
        [ 4]])
torch.Size([4, 1])
tensor([[ 1],
        [22],
        [11],
        [ 4]])
tensor([18,  1, 21,  7])
tensor([[ 4],
        [14],
        [ 7],
        [20]])
torch.Size([4, 1])
tensor([[ 4],
        [14],
        [ 7],
        [20]])
tensor([21, 26,  7, 12])
tensor([[ 7],
        [12],

tensor([[11],
        [22],
        [ 4],
        [25]])
tensor([ 0,  8, 14, 15])
tensor([[13],
        [21],
        [ 0],
        [ 1]])
torch.Size([4, 1])
tensor([[13],
        [21],
        [ 0],
        [ 1]])
tensor([ 2, 26,  7, 18])
tensor([[15],
        [12],
        [20],
        [ 4]])
torch.Size([4, 1])
tensor([[15],
        [12],
        [20],
        [ 4]])
tensor([10, 13,  3, 26])
tensor([[23],
        [26],
        [16],
        [12]])
torch.Size([4, 1])
tensor([[23],
        [26],
        [16],
        [12]])
tensor([26, 24, 24,  4])
tensor([[12],
        [10],
        [10],
        [17]])
torch.Size([4, 1])
tensor([[12],
        [10],
        [10],
        [17]])
tensor([18, 25,  7,  5])
tensor([[ 4],
        [11],
        [20],
        [18]])
torch.Size([4, 1])
tensor([[ 4],
        [11],
        [20],
        [18]])
tensor([14, 17,  0, 14])
tensor([[ 0],
        [ 3],
        [13],
        [ 0]])
torch.Size([4, 1])
tensor([[ 0],
        [ 3],
        [13],
        [ 

...

tensor([ 9,  3, 11, 20])
tensor([[22],
        [16],
        [24],
        [ 6]])
torch.Size([4, 1])
tensor([[22],
        [16],
        [24],
        [ 6]])
tensor([25,  8, 18, 22])
tensor([[11],
        [21],
        [ 4],
        [ 8]])
torch.Size([4, 1])
tensor([[11],
        [21],
        [ 4],
        [ 8]])
tensor([ 3, 26,  5, 13])
tensor([[16],
        [12],
        [18],
        [26]])
torch.Size([4, 1])
tensor([[16],
        [12],
        [18],
        [26]])
tensor([ 5,  4, 16, 26])
tensor([[18],
        [17],
        [ 2],
        [12]])
torch.Size([4, 1])
tensor([[18],
        [17],
        [ 2],
        [12]])
tensor([13,  8, 11, 23])
tensor([[26],
        [21],
        [24],
        [ 9]])
torch.Size([4, 1])
tensor([[26],
        [21],
        [24],
        [ 9]])
tensor([ 9, 23, 21,  8])
tensor([[22],
        [ 9],
        [ 7],
        [21]])
torch.Size([4, 1])
tensor([[22],
        [ 9],
        [ 7],
        [21]])
tensor([ 3, 12,  8,  1])
tensor([[16],
        [25],

tensor([ 1, 10, 21, 25])
tensor([[14],
        [23],
        [ 7],
        [11]])
torch.Size([4, 1])
tensor([[14],
        [23],
        [ 7],
        [11]])
tensor([ 3,  9,  0, 15])
tensor([[16],
        [22],
        [13],
        [ 1]])
torch.Size([4, 1])
tensor([[16],
        [22],
        [13],
        [ 1]])
tensor([26,  2,  3, 21])
tensor([[12],
        [15],
        [16],
        [ 7]])
torch.Size([4, 1])
tensor([[12],
        [15],
        [16],
        [ 7]])
tensor([11,  6, 22, 10])
tensor([[24],
        [19],
        [ 8],
        [23]])
torch.Size([4, 1])
tensor([[24],
        [19],
        [ 8],
        [23]])
tensor([ 0, 14,  5,  8])
tensor([[13],
        [ 0],
        [18],
        [21]])
torch.Size([4, 1])
tensor([[13],
        [ 0],
        [18],
        [21]])
tensor([13, 13, 17, 11])
tensor([[26],
        [26],
        [ 3],
        [24]])
torch.Size([4, 1])
tensor([[26],
        [26],
        [ 3],
        [24]])
tensor([ 7, 23,  6,  3])
tensor([[20],
        [ 9],

tensor([ 6,  5, 14, 12])
tensor([[19],
        [18],
        [ 0],
        [25]])
torch.Size([4, 1])
tensor([[19],
        [18],
        [ 0],
        [25]])
tensor([21, 22,  0,  2])
tensor([[ 7],
        [ 8],
        [13],
        [15]])
torch.Size([4, 1])
tensor([[ 7],
        [ 8],
        [13],
        [15]])
tensor([22, 12, 12, 21])
tensor([[ 8],
        [25],
        [25],
        [ 7]])
torch.Size([4, 1])
tensor([[ 8],
        [25],
        [25],
        [ 7]])
tensor([ 5,  7, 22, 23])
tensor([[18],
        [20],
        [ 8],
        [ 9]])
torch.Size([4, 1])
tensor([[18],
        [20],
        [ 8],
        [ 9]])
tensor([18,  1,  3, 20])
tensor([[ 4],
        [14],
        [16],
        [ 6]])
torch.Size([4, 1])
tensor([[ 4],
        [14],
        [16],
        [ 6]])
tensor([25,  9, 18, 12])
tensor([[11],
        [22],
        [ 4],
        [25]])
torch.Size([4, 1])
tensor([[11],
        [22],
        [ 4],
        [25]])
tensor([ 0,  8, 14, 15])
tensor([[13],
        [21],

tensor([10, 23, 17,  2])
tensor([[23],
        [ 9],
        [ 3],
        [15]])
torch.Size([4, 1])
tensor([[23],
        [ 9],
        [ 3],
        [15]])
tensor([12,  6,  8,  3])
tensor([[25],
        [19],
        [21],
        [13]])
torch.Size([4, 1])
tensor([[25],
        [19],
        [21],
        [16]])
tensor([14,  7,  7, 13])
tensor([[ 0],
        [20],
        [20],
        [26]])
torch.Size([4, 1])
tensor([[ 0],
        [20],
        [20],
        [26]])
tensor([25,  6, 24, 18])
tensor([[11],
        [19],
        [10],
        [ 4]])
torch.Size([4, 1])
tensor([[11],
        [19],
        [10],
        [ 4]])
tensor([19, 25,  2,  0])
tensor([[ 5],
        [11],
        [15],
        [13]])
torch.Size([4, 1])
tensor([[ 5],
        [11],
        [15],
        [13]])
tensor([ 2, 15, 17,  3])
tensor([[15],
        [ 1],
        [ 3],
        [16]])
torch.Size([4, 1])
tensor([[15],
        [ 1],
        [ 3],
        [16]])
tensor([10, 11, 21,  6])
tensor([[23],
        [24],

tensor([18,  1, 21,  7])
tensor([[ 4],
        [14],
        [ 7],
        [20]])
torch.Size([4, 1])
tensor([[ 4],
        [14],
        [ 7],
        [20]])
tensor([21, 26,  7, 12])
tensor([[ 7],
        [12],
        [20],
        [25]])
torch.Size([4, 1])
tensor([[ 7],
        [12],
        [20],
        [25]])
tensor([ 5, 11, 11,  7])
tensor([[18],
        [24],
        [24],
        [20]])
torch.Size([4, 1])
tensor([[18],
        [24],
        [24],
        [20]])
tensor([ 5, 17,  7, 25])
tensor([[18],
        [ 3],
        [20],
        [11]])
torch.Size([4, 1])
tensor([[18],
        [ 3],
        [20],
        [11]])
tensor([ 2,  1, 20, 13])
tensor([[15],
        [14],
        [ 6],
        [26]])
torch.Size([4, 1])
tensor([[15],
        [14],
        [ 6],
        [26]])
tensor([17, 15, 13,  3])
tensor([[ 3],
        [ 1],
        [26],
        [16]])
torch.Size([4, 1])
tensor([[ 3],
        [ 1],
        [26],
        [16]])
epoch: 14 loss: 14.975132093733304
